In [13]:
import torch
from torch.nn import functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR  #
# from torch.utils.data import TensorDataset, \
#     DataLoader  # Own stuff set of data quality checks, tensor shapes might be different dataloader loads the tensor,
import pytorch_lightning as pl  #
from pytorch_lightning.callbacks.early_stopping import \
    EarlyStopping  # early stop when you reach optimum loss, 3 times in a row gradient descent
from pytorch_lightning.callbacks import LearningRateMonitor  # delta (loss / accuracy)
from pytorch_lightning.loggers import MLFlowLogger  # Model tracking
import torch
import torch.nn as nn
from transformers import DistilBertModel
from data_preprocessing import injestDataset, downSample
from data_cleaning import feature_cleaner, cleaningPreprocess
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer, LabelBinarizer
from sklearn.compose import ColumnTransformer

import torchmetrics
import pytorch_lightning as pl
from transformers import BertTokenizer
# from transformers import DistilBertModel

In [14]:
torch.set_default_dtype(torch.float32)

In [15]:
data = injestDataset()

In [16]:
df = downSample(data,"Recommended IND",1000)

In [17]:
df.dtypes

trueIndex           int64
Review Text        object
Rating              int64
Recommended IND     int64
dtype: object

In [18]:
df = cleaningPreprocess(df, "Review Text")

In [19]:
df = df.rename(columns={"Recommended IND": "targetOne", "Review Text": "feature", "Rating": "targetTwo"})

In [20]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [21]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [22]:
# Here is your dataset
class MyDataset(Dataset):
    def __init__(self, df,transform=None):
        self.input_data = df
        # self.max_length = 30
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenize_dataframe()
        self.train_val_test_idx()
        self.split_columns()
        self.targertOne_labelEncoder()
        self.targertTwo_labelEncoder()

    def tokenize_dataframe(self):
        tokenized_texts = []
        input_ids = []
        attention_masks = []

        for text in self.input_data['feature']:
            encoded_inputs = self.tokenizer(
                text,
                add_special_tokens=True,
                # max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            tokenized_texts.append(encoded_inputs)
            input_ids.append(encoded_inputs['input_ids'])
            attention_masks.append(encoded_inputs['attention_mask'])
        
        self.input_data['tokenizedFeature'] = tokenized_texts
        self.input_data['input_ids'] = input_ids
        self.input_data['attention_mask'] = attention_masks


    def split_columns(self):
        self.features = self.input_data["feature"]
        self.targetOne = self.input_data["targetOne"]
        self.targetTwo = self.input_data["targetTwo"]

            
    def train_val_test_idx(self):
        idx = list(range(len(self.input_data)))
        train_idx, val_test_idx = train_test_split(
                                idx,
                                train_size=0.8,
                                stratify=self.input_data["targetOne"],
                                random_state=500)
                                
        test_idx,  val_idx, = train_test_split(
                                val_test_idx,
                                train_size=0.5,
                                random_state=500)

        self.train_idx = train_idx 
        self.test_idx = test_idx
        self.val_idx = val_idx,
    
    def targertOne_labelEncoder(self):
        self.targetOne_label_binarizer = MultiLabelBinarizer()
        self.input_data_2 =  self.targetOne_label_binarizer.fit_transform(self.input_data["targetOne"].astype("str"))

    def targertTwo_labelEncoder(self):
        self.targetTwo_label_binarizer = MultiLabelBinarizer()
        self.input_data_3 =  self.targetTwo_label_binarizer.fit_transform(self.input_data["targetTwo"].astype("str"))

    # class to_tensor:
    def conver_to_tensor(self):
        
        return torch.tensor(features,dtype=torch.float32,device='mps:0'), torch.tensor(targets ,dtype=torch.int32,  device='mps:0')

        
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, index):
        sample = self.input_data.iloc[index]
        return sample


In [23]:
dataset = MyDataset(df)

In [126]:
import pandas as pd
import numpy as np

# Sample 2D array
two_dimensional_array = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# x = torch.tensor(two_dimensional_array)
# Transpose the 2D array
first_elements = [subarray for subarray in np.split(two_dimensional_array,len(two_dimensional_array))]

df = pd.DataFrame({'Single_Column': first_elements})
df
df["neww"] = first_elements
# df["neww_s"] = list(x)


In [130]:
torch.tensor(df['neww'])

tensor([[[1, 2, 3]],

        [[4, 5, 6]],

        [[7, 8, 9]]])

In [125]:
len(two_dimensional_array)

3

[array([[1, 2, 3]]), array([[4, 5, 6]]), array([[7, 8, 9]])]

In [106]:
list(x)[1]

tensor([[4, 5, 6]])

In [119]:
type(x)

torch.Tensor

In [109]:
df

,Single_Column,neww,neww_s
0,"[[tensor(1), tensor(2), tensor(3)]]","[[tensor(1), tensor(2), tensor(3)]]","[[tensor(1), tensor(2), tensor(3)]]"
1,"[[tensor(4), tensor(5), tensor(6)]]","[[tensor(4), tensor(5), tensor(6)]]","[[tensor(4), tensor(5), tensor(6)]]"
2,"[[tensor(7), tensor(8), tensor(9)]]","[[tensor(7), tensor(8), tensor(9)]]","[[tensor(7), tensor(8), tensor(9)]]"


In [66]:
df.iloc[[1]]

,Single_Column
1,"[tensor(4), tensor(5), tensor(6)]"


In [62]:
two_dimensional_array = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

In [65]:
two_dimensional_array[1]

[4, 5, 6]

In [71]:
two_dimensional_array.

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [81]:
import pandas as pd
import numpy as np
import torch

# Sample 2D array
two_dimensional_array = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# Convert the 2D array to a list of PyTorch tensors
tensors_list = [subarray for subarray in two_dimensional_array]

# Convert the list of PyTorch tensors to a list of NumPy arrays
# numpy_arrays_list = [tensor.numpy() for tensor in tensors_list]

# Create a DataFrame with a single column containing NumPy arrays
df = pd.DataFrame({'Single_Column': tensors_list})

print(df)


  Single_Column
0     [1, 2, 3]
1     [4, 5, 6]
2     [7, 8, 9]


In [79]:
df.iloc[1]

Single_Column    [tensor(4), tensor(5), tensor(6)]
Name: 1, dtype: object

In [64]:
torch.tensor(two_dimensional_array)[1]

tensor([4, 5, 6])

In [50]:
df

,Single_Column
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9


In [46]:
df 

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9


In [38]:
pd.DataFrame(dataset.input_data_2)

,0,1
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
995,1,0
996,1,0
997,1,0
998,1,0


array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [24]:
dataset[1]["input_ids"].size()[1]## Double bracket for dataframe if single value

512

In [25]:
dataset[[1]]

,trueIndex,feature,targetTwo,targetOne,tokenizedFeature,input_ids,attention_mask
1,4578,great spring time dress looks good thin top ca...,5,1,"[input_ids, token_type_ids, attention_mask]","[[tensor(101), tensor(2307), tensor(3500), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


In [26]:
def dataLoader():
    trainDataLoader = DataLoader(dataset[dataset.train_idx], batch_size=10, shuffle=True)
    testDataLoader = DataLoader(dataset[dataset.test_idx], batch_size=10, shuffle=True)
    valDataLoader = DataLoader(dataset[dataset.val_idx], batch_size=10, shuffle=True)
    return trainDataLoader, testDataLoader, valDataLoader

In [27]:
x, y, z = dataLoader()

In [28]:
class MultiClassClassifier(pl.LightningModule):
    def __init__(self, seed, output_dim, input_dim=4, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
        # lower than epoch higher than iterations
        super().__init__()
        # self.save_hyperparameters()  # save_hyperparameters
        pl.seed_everything(seed)
        self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.pre_classifier = torch.nn.Linear(512, 512)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(512,output_dim)

    def forward(self, batch):
        # input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
        # input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
        embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'],return_dict=False)[0]
        pooler = self.pre_classifier(embeddings)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, self.hparams.total_steps)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        logits = self.forward(batch)
        loss = F.nll_loss(logits, batch['labels'])
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        logits = self.forward(batch)
        loss = F.nll_loss(logits, batch['labels'])
        self.log("val_loss", loss)

    def test_step(self, batch, batch_idx):
        logits = self.forward(batch)
        loss = F.nll_loss(logits, batch['labels'])
        self.log("test_loss", loss)

    def predict(self, x):
        # TODO this will be changed
        self.eval()
        logits = self.forward(x)
        self.train()
        return torch.argmax(logits, dim=1).detach().numpy()

In [29]:

# Here is your PyTorch Lightning model
class LitNetwork(pl.LightningModule):
    def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
        super(LitNetwork, self).__init__()
        self.train_ds = trainDataLoader
        self.test_ds = testDataLoader
        self.val_ds = valDataLoader
        self.linear1 = nn.Linear(48, 25)
        self.linear2 = nn.Linear(25, 2)
        # self.label_binarizer = trainDataLoader.

    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = self.linear2(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        
        # Calculate accuracy
        preds = torch.argmax(y_hat, dim=1)
        acc = torchmetrics.functional.accuracy(preds,  y)

        # Logging to TensorBoard by default
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        return self.train_ds

    def test_dataloader(self):
        return self.test_ds  # return your dataloader
        
    def val_dataloader(self):
        return self.val_ds  # return your dataloader

In [30]:
# class MultiClassClassifier(pl.LightningModule):
#     def __init__(self, seed, output_dim, input_dim=4, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
#         # lower than epoch higher than iterations
#         super().__init__()
#         self.save_hyperparameters()  # save_hyperparameters
#         pl.seed_everything(seed)
#         self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         # self.pre_classifier = torch.nn.Linear(768, 768)
#         # self.dropout = torch.nn.Dropout(0.3)
#         # self.classifier = torch.nn.Linear(768,output_dim)

#     def forward(self, batch):
#         # input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
#         # input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
#         embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'],return_dict=False)[0]
#         pooler = self.pre_classifier(embeddings)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

#     def configure_optimizers(self):
#         optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate)
#         scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, self.hparams.total_steps)
#         return [optimizer], [scheduler]

#     def training_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("train_loss", loss, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("test_loss", loss)

#     def predict(self, x):
#         # TODO this will be changed
#         self.eval()
#         logits = self.forward(x)
#         self.train()
#         return torch.argmax(logits, dim=1).detach().numpy()

#     # def df_to_tensor(self, df, target_col=None, format=np.float32):
#     #     if target_col is not None:
#     #         feature_cols = [col for col in df.columns if col != target_col]
#     #         tensor = TensorDataset(
#     #             torch.tensor(df[feature_cols].values.astype(format)),
#     #             torch.tensor(df[target_col].values),
#     #         )
#     #     else:
#     #         tensor = torch.tensor(df.values.astype(format))
#     #     return tensor

#     def tensor_to_loader(self, tensor, batch_size, num_workers, shuffle=True):
#         loader = DataLoader(dataset=tensor, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
#         return loader

#     def setup_trainer(self, experiment, run_id, max_epochs):
#         # mlf_logger = MLFlowLogger(experiment_name=experiment)
#         # mlf_logger._run_id = run_id
#         trainer = pl.Trainer(
#             max_epochs=max_epochs,
#             callbacks=[EarlyStopping(monitor="val_loss"), LearningRateMonitor(logging_interval="epoch")],
#             # logger=mlf_logger,
#         )

#         return trainer

In [31]:
# class CustomBertClassifier(nn.Module):
#     def __init__(self, bert_model):
#         super(CustomBertClassifier, self).__init__()
#         self.bert = bert_model
#         self.fc = nn.Linear(bert_model.config.hidden_size, 2)  # Binary classification

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids, attention_mask=attention_mask)
#         pooled_output = outputs['pooler_output']
#         logits = self.fc(pooled_output)
#         return logits

In [32]:

# # Here is your PyTorch Lightning model
# class LitNetwork(pl.LightningModule):
#     def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
#         super(LitNetwork, self).__init__()
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.linear1 = nn.Linear(48, 25)
#         self.linear2 = nn.Linear(25, 2)
#         # self.label_binarizer = trainDataLoader.

#     def forward(self, x):
#         x = torch.sigmoid(self.linear1(x))
#         x = self.linear2(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.functional.cross_entropy(y_hat, y)
        
#         # Calculate accuracy
#         preds = torch.argmax(y_hat, dim=1)
#         acc = torchmetrics.functional.accuracy(preds,  y)

#         # Logging to TensorBoard by default
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader